## 04.토치텍스트(TorchText)의 batch_first

이번 챕터에서는 토치텍스트에서 배치퍼스트(batch_first)를 True로 한 경우와 False를 한 경우를 비교해보겠다. 이번 챕터는 토치텍스트 튜토리얼 챕터가 아니므로, 토치텍스트를 이전 챕터를 통해 이미 알고 있다고 가정한다.



### 1.훈련 데이터와 테스트 데이터로 분리하기



In [1]:
import urllib.request
import pandas as pd


우선 인터넷에서 IMDB 리뷰 데이터를 다운로드 받는다.

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv",
                           filename="IMDb_Reviews.csv")


('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f292da2a780>)

In [3]:
ls

IMDb_Reviews.csv  sample_data/


다운로드한 IMDB 리뷰 데이터를 데이터프레임에 저장한다.


In [4]:
df = pd.read_csv('IMDb_Reviews.csv', encoding = 'latin1')

전체 샘플의 개수를 보겠다.

In [5]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


전체 샘플의 개수는 50,000개이다. 25,000개씩 분리하여 훈련 데이터와 테스트 데이터로 분리해보겠다.


In [6]:
train_df = df[:25000]
test_df = df[25000:]

상위 25,000개의 행은 train_df에, 하위 25,000개의 행은 test_df에 저장하였다.

In [7]:
train_df.to_csv('train_data.csv', index = False)
test_df.to_csv('test_data.csv', index = False)

index = False를 하면 인덱스를 저장하지 않는다.


### 2.필드 정의하기(torchtext.data)

torchtext.data를 통해 필드(Field)를 정의한다.


In [8]:
from torchtext import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = str.split,
                  lower = True,
                  batch_first = True, # <-- 이 부분을 True로 한다.
                  fix_length = 20)

LABEL = data.Field(sequential = False,
                   use_vocab = False,
                   batch_first = False,
                   is_target = True)

* batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)

batch_first 인자를 True로 한다.


### 3.데이터셋 / 단어 집합 / 데이터로더 만들기



In [14]:
from torchtext.data import TabularDataset
from torchtext.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행한다.
train_data, test_data = TabularDataset.splits(
    path = '.', train = 'train_data.csv', test = 'test_data.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)], 
    skip_header = True
)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성한다.
TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠다.
batch_size = 5
train_loader = Iterator(dataset = train_data,
                        batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [15]:
print(batch.text)

tensor([[  10,   10,   50, 5296,    9],
        [   7,   20,  270, 1349,  133],
        [   3,   14,  239, 2704,   26],
        [ 229,    3,   82,    0,  617],
        [1093, 2359,    0,   13,    9],
        [  20, 1168,    9,  721,   14],
        [  42,   76,   14, 1420,    8],
        [   3,    9,   22,   25, 1255],
        [1862,  144,    8,  448,   17],
        [  12,  518,    3, 1192,    0],
        [2253, 1257,    0, 3497, 1099],
        [   3,    9, 8362,    0,    0],
        [   0,  144,   17,    4,   15],
        [3369,  115,    2,    2,    2],
        [   4,   12,    0,  124,  451],
        [ 197,   46, 5069,  422, 1064],
        [2458,    5,    3, 2834,  924],
        [   0,    2, 6048,    0,  486],
        [  50,  181,    5,    9,    0],
        [  27,   26, 4916,   37,  805]])


배치 크기가 5이기 때문에 5개의 샘플이 출력되는 것을 볼 수 있다. 각 샘플의 길이는 20의 길이는 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문이다. 다시 말해 하나의 미니 배치의 크기는 (배치 크기 x fix_length)이다.

미니 배치의 크기를 출력해보자.


In [16]:
print(batch.text.shape)

torch.Size([20, 5])


이제 batch_first를 False로 한 경우를 보겠다.


### 4.필드 재정의하기(torchtext.data)



In [13]:
# 필드 정의
TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = str.split,
                  lower = True,
                  fix_length = 20)

LABEL = data.Field(sequential = False,
                   use_vocab = False,
                   batch_first = False,
                   is_target = True)

TEXT 필드에서 batch_first = True 인자를 제거하였다. 기본값이 False이므로 batch_first의 값이 False가 된다.


### 5.batch_first = False로 하였을 경우의 텐서 크기

필드를 재정의하였으므로 데이터셋, 단어 집합, 데이터로더를 만드는 과정을 다시 수행해주어야 한다. 

**3. 데이터셋 / 단어 집합 / 데이터로더 만들기의 코드를 재수행했다고 가정한다.**
(위의 코드블럭 다시 실행)


In [18]:
# 첫번째 미니 배치 출력
print(batch.text)

tensor([[  10,   10,   50, 5296,    9],
        [   7,   20,  270, 1349,  133],
        [   3,   14,  239, 2704,   26],
        [ 229,    3,   82,    0,  617],
        [1093, 2359,    0,   13,    9],
        [  20, 1168,    9,  721,   14],
        [  42,   76,   14, 1420,    8],
        [   3,    9,   22,   25, 1255],
        [1862,  144,    8,  448,   17],
        [  12,  518,    3, 1192,    0],
        [2253, 1257,    0, 3497, 1099],
        [   3,    9, 8362,    0,    0],
        [   0,  144,   17,    4,   15],
        [3369,  115,    2,    2,    2],
        [   4,   12,    0,  124,  451],
        [ 197,   46, 5069,  422, 1064],
        [2458,    5,    3, 2834,  924],
        [   0,    2, 6048,    0,  486],
        [  50,  181,    5,    9,    0],
        [  27,   26, 4916,   37,  805]])


하나의 미니 배치의 크기는 (fix_length x 배치 크기)이다.


In [19]:
print(batch.text.shape)

torch.Size([20, 5])
